# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.tool import meta as ml_meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

# Set Information

In [2]:
 # regression_1 --> regression
# regression_2 --> forecasting
# classification
model_mode = "regression_1"

In [3]:
if model_mode == "regression_1":
    ms_name_X = 'regression_energy_cleanLevel0_trainX'
    ms_name_y = 'regression_energy_cleanLevel0_trainy'
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="windows_split"
    data_y_flag = 'true' # 이미 만들어진 Y 데이터를 활용함
    
    model_name = 'Regression_API_TEST'
    model_method = 'LSTM_rg'
    model_purpose = 'Regression'
    
    
elif model_mode == "regression_2":
    ms_name_X = 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX'
    ms_name_y = 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainy'
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'
    data_y_flag = False # Y데이터는 없음, X 에서 Y 데이터를 도출함
    
    model_name = 'Forecasting_API_TEST'
    model_method = 'LSTM_rg'
    model_purpose = 'Regression'
    
    
elif model_mode == "classification":
    ms_name_X = 'classification_actionPattern_cleanLevel0_trainX'
    ms_name_y = 'classification_actionPattern_cleanLevel0_trainy'
    feature_X_list = ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
    feature_y_list = ['value']
    split_mode = 'windows_split'
    data_y_flag = 'true'
    
    model_name = 'Classification_API_TEST'
    model_method = 'LSTM_cf'
    model_purpose = 'Classification'
    

In [4]:
model_mode

'regression_1'

In [5]:
if model_mode == "regression_1" or model_mode == "regression_2":
    print("1")
    regression_model_info ={"LSTM_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                         "GRU_rg":{"hidden_size":64,"num_layers":2,"output_dim":1,"dropout":0.1,"bidirectional":True},
                         "CNN_1D_rg":{"output_channels":64,"kernel_size":3,"stride":1,"dropout":0.1,"padding":0},
                         "LSTM_FCNs_rg":{"num_layers":2,"lstm_dropout":0.4,"fc_dropout":0.1},
                         "FC_rg":{"dropout":2,"bias":0.4}}
    model_parameter = regression_model_info[model_method]
    train_parameter = {"lr":1e-4,"weight_decay":1e-6,"n_epochs":10,"batch_size":16}

elif model_mode == "classification":
    print("2")
    classification_model_info ={"LSTM_cf":{"hidden_size":64,"num_layers":2,"num_classes":2,"dropout":0.1,"bidirectional":True},
                         "GRU_cf":{"hidden_size":64,"num_layers":2,"num_classes":2,"dropout":0.1,"bidirectional":True},
                         "CNN_1D_cf":{"output_channels":64,"kernel_size":3,"stride":1,"dropout":0.1,"padding":0,"num_classes":6},
                         "LSTM_FCNs_cf":{"num_layers":2,"lstm_dropout":0.4,"fc_dropout":0.1,"num_classes":6},
                         "FC_cf":{"dropout":0.1,"bias":True}}
    model_parameter = classification_model_info[model_method]
    train_parameter = {"lr":1e-4,"weight_decay":1e-6,"n_epochs":10,"batch_size":16}

1


# Set Parameter

In [6]:
model_name = 'api_test_regression_version'

In [7]:
params = {
    'bk_name_X' : 'integration',
    'ms_name_X' : ms_name_X,
    'feature_X_list' : feature_X_list,
    'bk_name_y' : 'integration',
    'ms_name_y' : ms_name_y,
    'feature_y_list' : feature_y_list,
    'data_y_flag' : data_y_flag,
    'data_clean_option' : False,
    'model_purpose' : model_purpose,
    'model_method' : model_method,
    'model_name' : model_name,
    'model_tags' : 'tagstest',
    'train_parameter' : train_parameter,
    'model_parameter' : model_parameter,
    'scaler_param':'scale'
}

if model_mode == "regression_2":
    params['future_step'] = 2
    params['past_step'] = 24

In [8]:
params['data_y_flag']

'true'

# Training

## Use Function in API

In [9]:
from KETIAppDataServer2.model import dbModel
dbModel.DB_init()
from KETIAppDataServer2.API.machine_learning.model_train import set_train_data, training, update_meta_data, check_model_name

cpu is available.


In [10]:
# 1. Model Name Check
model_name = check_model_name(params['model_name'], [params['model_purpose'], params['model_method'],params['bk_name_X'],params['ms_name_X']])
params['model_name'] = model_name
data_info_meta = {}

# 2. data meta check
data_meta = mongodb_client.get_document_by_json(params['bk_name_X'], params['ms_name_X'], {'ms_name': params['ms_name_X']})
try:
    data_info_meta['train_data_info'] = data_meta[0]
except:
    data_info_meta['train_data_info'] = {}

# 3. Make Train Data
train_X_array, train_y_array, val_X_array, val_y_array, transform_parameter, scale_path = set_train_data(params)

# 4. Make data meta
data_info_meta = dict(data_info_meta, **params)
data_info_meta = update_meta_data(data_info_meta, 'transform_parameter', transform_parameter)
data_info_meta = update_meta_data(data_info_meta, 'scale_files', scale_path)

# 5. model trqining
train_parameter, model_parameter, model_path = training(train_X_array, train_y_array, val_X_array, val_y_array, params)

# 6. model meta
model_info_meta = {}
model_info_meta = update_meta_data(model_info_meta, 'train_parameter', train_parameter)
model_info_meta = update_meta_data(model_info_meta, 'model_parameter', model_parameter)
model_info_meta = update_meta_data(model_info_meta, 'model_files', model_path)

# 7. save meta
result = dict(data_info_meta, **model_info_meta)
ml_meta.save_model_meta_data(mongodb_client, result)

['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File
window_size: 144 nan_limit_num: 72
(10944, 24) (76, 144, 24)
(76, 1) (76, 1)
window_size: 144 nan_limit_num: 72
(2736, 24) (19, 144, 24)
(19, 1) (19, 1)
{'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': True}
./Models/LSTM_rg/api_test_regression_version/regression_energy_cleanLevel0_trainX/d531eec11664669cff1f6a3ad9639012/model.pkl
Start training model
[1/10] Training loss: 0.1412	 Validation loss: 0.0904
[2/10] Training loss: 0.1274	 Validation loss: 0.0842
[3/10] Training loss: 0.1296	 Validation loss: 0.0780
[4/10] Training loss: 0.1162	 Validation loss: 0.0721
[5/10] Training loss: 0.1044	 Validation loss: 0.0665
[6/10] Training loss: 0.0935	 Validation loss: 0.0612
[7/10] Training loss: 0.0980	 V

200

In [11]:
result

{'train_data_info': {'bucket_name': 'integration',
  'collection_name': 'regression_energy',
  'ms_name': 'regression_energy_cleanLevel0_trainX',
  'ingestion_type': 'multiple_ms_by_time',
  'ingestion_param': {'ms_list_info': [['life_indoor_environment',
     'humidityTrain_10min'],
    ['life_indoor_environment', 'temperatureTrain_10min'],
    ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
   'start_time': '2016-01-11',
   'end_time': '2016-04-15'},
  'processing_type': 'step_3',
  'process_param': {'refine_param': {'remove_duplication': {'flag': False},
    'static_frequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certain_error_to_NaN': {'flag': False},
    'uncertain_error_to_NaN': {'flag': False}},
   'imputation_param': {'flag': False}},
  'integration_param': {'integration_frequency': 600,
   'param': {},
   'method': 'meta',
   'integration_duration': 'common'},
  'clean_level': 0},
 'bk_name_X': 'integration',
 'ms_name_X': 'regres

In [12]:
print(data_info_meta)
print("----------------------------------------------------------------------------------------")

{'train_data_info': {'bucket_name': 'integration', 'collection_name': 'regression_energy', 'ms_name': 'regression_energy_cleanLevel0_trainX', 'ingestion_type': 'multiple_ms_by_time', 'ingestion_param': {'ms_list_info': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']], 'start_time': '2016-01-11', 'end_time': '2016-04-15'}, 'processing_type': 'step_3', 'process_param': {'refine_param': {'remove_duplication': {'flag': False}, 'static_frequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certain_error_to_NaN': {'flag': False}, 'uncertain_error_to_NaN': {'flag': False}}, 'imputation_param': {'flag': False}}, 'integration_param': {'integration_frequency': 600, 'param': {}, 'method': 'meta', 'integration_duration': 'common'}, 'clean_level': 0}, 'bk_name_X': 'integration', 'ms_name_X': 'regression_energy_cleanLevel0_trainX', 'feature_X_list': ['Press_m

In [13]:
print(model_info_meta)
print("$####################################################################################")

{'train_parameter': {'lr': 0.0001, 'weight_decay': 1e-06, 'device': 'cpu', 'n_epochs': 10, 'batch_size': 16}, 'model_parameter': {'rnn_type': 'lstm', 'input_size': 24, 'hidden_size': 64, 'num_layers': 2, 'output_dim': 1, 'dropout': 0.1, 'bidirectional': False}, 'model_files': {'modelFile': {'fileName': 'model.pth', 'filePath': './Models/LSTM_rg/api_test_regression_version/regression_energy_cleanLevel0_trainX/d531eec11664669cff1f6a3ad9639012/model.pkl'}}}
$####################################################################################


In [14]:
print(result)
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

{'train_data_info': {'bucket_name': 'integration', 'collection_name': 'regression_energy', 'ms_name': 'regression_energy_cleanLevel0_trainX', 'ingestion_type': 'multiple_ms_by_time', 'ingestion_param': {'ms_list_info': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']], 'start_time': '2016-01-11', 'end_time': '2016-04-15'}, 'processing_type': 'step_3', 'process_param': {'refine_param': {'remove_duplication': {'flag': False}, 'static_frequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certain_error_to_NaN': {'flag': False}, 'uncertain_error_to_NaN': {'flag': False}}, 'imputation_param': {'flag': False}}, 'integration_param': {'integration_frequency': 600, 'param': {}, 'method': 'meta', 'integration_duration': 'common'}, 'clean_level': 0}, 'bk_name_X': 'integration', 'ms_name_X': 'regression_energy_cleanLevel0_trainX', 'feature_X_list': ['Press_m